## Shapley effects for RC and $\theta_{11}$

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
from functools import partial
from ruspy.estimation.estimation import estimate
#from ruspy.simulation.simulation import simulate
from ruspy.model_code.fix_point_alg import calc_fixp
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.estimation.estimation_transitions import create_transition_matrix
from ruspy.model_code.demand_function import get_demand 
from python.econsa_shapley import get_shapley
from python.econsa_shapley import _r_condmvn
from python.sampling import get_cov_and_mean_rc_theta_32
from python.sampling import approx_comp_time
#from python.conditional_sampling import x_all
#from python.conditional_sampling import x_cond

In [2]:
run_simulation = False

In [3]:
# Set simulating variables.
disc_fac = 0.9999
num_buses = 50
num_periods = 120
gridsize = 1000
# We use the cost parameters and transition probabilities from the replication.
params = np.array([10.07780762, 2.29417622])
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
scale = 1e-3

init_dict_simulation = {
    "simulation": {
        "discount_factor": disc_fac,
        "periods": num_periods,
        "seed": 123,
        "buses": num_buses,
    },
    "plot": {"gridsize": gridsize},
}

In [4]:
# Calcualte objects necessary for the simulation process. See documentation for details.
num_states = 200
costs = calc_obs_costs(num_states, lin_cost, params, scale)

trans_mat = create_transition_matrix(num_states, trans_probs)
ev = calc_fixp(trans_mat, costs, disc_fac)[0]

In [5]:
# Use one init_dict for get_demand() and estimate().
init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

In [6]:
%%time
if run_simulation == True:
    num_sim = 1000
    cov, mean = get_cov_and_mean_rc_theta_32(num_sim, ev, costs, trans_mat, init_dict_simulation, init_dict_estimation)
    np.save(os.path.join('data', 'cov_rc_theta_11'), cov)
    np.save(os.path.join('data', 'mean_rc_theta_11'), mean)
else:
    cov = np.load(os.path.join('data', 'cov_rc_theta_11.npy'))
    mean = np.load(os.path.join('data', 'mean_rc_theta_11.npy'))

Wall time: 5 ms


Define functions `rust_model()`, and function arguments.

In [7]:
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis=1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
        )

In [8]:
def rust_model(x):
    if method == 'exact':
        n_evaluations = n_output + np.math.factorial(n_inputs) * (n_inputs -1) * n_outer * n_inner
    elif method == 'random':
        n_evaluations = n_output + n_perms * (n_inputs -1) * n_outer * n_inner
        
    demand_inputs = np.zeros((n_evaluations, 5))
    demand_inputs[:, :3] = np.array([0.39189182, 0.59529371, 0.01281447])
    demand_inputs[:, 3:] = x[:, :]
    
    demand_output = np.zeros((n_evaluations, 1))
    
    # Second, try with list comprehension (do not need to define demand_output first).
    #demand_output = [get_demand(init_dict_estimation, demand_dict, demand_inputs[sample, :]).iloc[0]['demand'] 
    #                for sample in np.arange(n_evaluations)]
    
    # First try with for loop.
    #for sample in np.arange(n_evaluations):
    #    demand_params = demand_inputs[sample, :]
    #    demand_output[sample] = get_demand(init_dict_estimation, demand_dict, demand_params).iloc[0]['demand']
    
    get_demand_partial = partial(get_demand, init_dict=init_dict_estimation, demand_dict=demand_dict)
    def get_demand_mapping(x):
        return get_demand_partial(demand_params=x).iloc[0]['demand']
    
    demand_output = np.array(list(map(get_demand_mapping, demand_inputs)))
    
    return demand_output

In [9]:
# Need demand at certain value of RC only.  Note RC is scaled by 1e-03.
demand_dict = {
    "RC_lower_bound": 11.5,
    "RC_upper_bound": 11.5,
    "demand_evaluations": 1,
    "tolerance": 1e-10,
    "num_periods": num_periods,
    "num_buses": num_buses,
}

In [10]:
n_inputs = 2
method = 'exact'
np.random.seed(1234)

if method == 'exact':
    n_perms = None
elif method == 'random':
    n_perms = 10
n_output = 10**3
n_outer = 15
n_inner = 3

Run for large no. of MC runs.

In [11]:
n_output = 10**3
n_outer = 10**2
n_inner = 20
# Time in s for 100 model evaluations.
time_model_eval = 35
approx_comp_time(time_model_eval, method, n_inputs, n_perms, n_output, n_outer, n_inner)

 5000 model evaluations 
 approx.  0.4861111111111111 hours 
 approx. 29.166666666666668 minutes


In [12]:
%%time

exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 33min 15s


In [13]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.422186,0.29853,-0.162934,1.007305
theta_11,0.577814,0.29853,-0.007305,1.162934


Another try with even larger no.s

In [20]:
n_output = 10**3
n_outer = 10**2
n_inner = 40
# Time in s for 100 model evaluations.
time_model_eval = 37
approx_comp_time(time_model_eval, method, n_inputs, n_perms, n_output, n_outer, n_inner)

 9000 model evaluations 
 approx.  0.925 hours 
 approx. 55.5 minutes


In [18]:
%%time

exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 1h 3min 57s


In [19]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.413136,0.292131,-0.159441,0.985713
theta_11,0.586864,0.292131,0.014287,1.159441


In [17]:
def estim_shapley_rc_theta_11(estim_dict):
    n_output = estim_dict['n_output']
    n_outer = estim_dict['n_outer']
    n_inner = estim_dict['n_inner']
    out = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
    out.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)
    out['N_V'] = n_output
    out['N_O'] = n_outer
    out['N_I'] = n_inner
    out['time'] = timeit
    return out

In [18]:
list_num_sim = [{'n_output': 10**3, 'n_outer': 15, 'n_inner': 3}, 
                {'n_output': 10**3, 'n_outer': 15, 'n_inner': 5}, 
                {'n_output': 10**3, 'n_outer': 15, 'n_inner': 7}, 
                {'n_output': 10**3, 'n_outer': 20, 'n_inner': 3}, 
                {'n_output': 10**3, 'n_outer': 20, 'n_inner': 7}]

In [23]:
def sim_shapley():
    for i in list_num_sim:
        return estim_shapley_rc_theta_11(i)

In [12]:
%%time

exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 6min 44s


In [13]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.425934,0.301181,-0.164381,1.016249
theta_11,0.574066,0.301181,-0.016249,1.164381


Try with larger no. of MC runs.

In [29]:
%%time
n_output = 10**2
n_outer = 40
n_inner = 20
exact_shapley = get_shapley(method, rust_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)
exact_shapley.rename(index={'X1': 'RC', 'X2': 'theta_11'}, inplace=True)

Wall time: 11min 22s


In [30]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.437425,0.309306,-0.168815,1.043666
theta_11,0.562575,0.309306,-0.043666,1.168815


In [27]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
RC,0.386826,0.273527,-0.149287,0.922939
theta_11,0.613174,0.273527,0.077061,1.149287
